<a href="https://colab.research.google.com/github/brothermin00/JNU_2023/blob/main/unet_plot_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://github.com/onebottlekick/JNU_dl/releases/download/1/unet_checkpoints.data-00000-of-00001
!wget https://github.com/onebottlekick/JNU_dl/releases/download/1/unet_checkpoints.index
!wget https://github.com/onebottlekick/JNU_dl/releases/download/1/data.zip

In [ ]:
import zipfile

with zipfile.ZipFile('data.zip', 'r') as f:
    f.extractall('./')

In [ ]:
import os
import random
from PIL import Image

class DataReader():
    def __init__(self):
        self.label = ["Background", "Pet"]

        self.train_X = []
        self.train_Y = []
        self.test_X = []
        self.test_Y = []

        self.read_data()

    def read_data(self):
        print("Reading Data...")
        images = os.listdir("data/images")
        annotations = os.listdir("data/annotations")

        images.sort()
        annotations.sort()

        data = []

        for i in range(len(images)):
            img = Image.open("data/images/" + images[i])
            ant = Image.open("data/annotations/" + annotations[i])

            if img.mode != "RGB":
                img = img.convert("RGB")

            X = np.asarray(img) / 255.0

            Y_temp = np.asarray(ant)[:, :, 0]
            Y = np.zeros_like(Y_temp)
            Y[Y_temp > 127.5] = 1.0

            data.append((X, Y))
            img.close()
            ant.close()

        random.shuffle(data)

        for i, el in enumerate(data):
            if i < 0.8*len(data):
                self.train_X.append(el[0])
                self.train_Y.append(el[1])
            else:
                self.test_X.append(el[0])
                self.test_Y.append(el[1])

        self.train_X = np.asarray(self.train_X)
        self.train_Y = np.asarray(self.train_Y)
        self.test_X = np.asarray(self.test_X)
        self.test_Y = np.asarray(self.test_Y)

        # 데이터 읽기가 완료되었습니다.
        # 읽어온 데이터의 정보를 출력합니다.
        print("\n\nData Read Done!")
        print("Training X Size : " + str(self.train_X.shape))
        print("Training Y Size : " + str(self.train_Y.shape))
        print("Test X Size : " + str(self.test_X.shape))
        print("Test Y Size : " + str(self.test_Y.shape) + '\n\n')

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sys

np.set_printoptions(linewidth=np.inf)


# 인공신경망을 제작합니다.
def graph(input_X, input_Y):
    input = keras.layers.Input((input_X, input_Y, 3))

    # 첫 번째 Convolution Block
    Conv1 = keras.layers.Conv2D(16, (3, 3), activation="relu", padding='same')(input)
    Conv1 = keras.layers.Conv2D(16, (3, 3), activation="relu", padding='same')(Conv1)
    Pool1 = keras.layers.MaxPooling2D((2, 2)) (Conv1)

    # 두 번째 Convolution Block
    Conv2 = keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same')(Pool1)
    Conv2 = keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same')(Conv2)
    Pool2 = keras.layers.MaxPooling2D((2, 2))(Conv2)

    # 세 번째 Convolution Block
    Conv3 = keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same')(Pool2)
    Conv3 = keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same')(Conv3)
    Pool3 = keras.layers.MaxPooling2D((2, 2))(Conv3)

    # 네 번째 Convolution Block
    Conv4 = keras.layers.Conv2D(128, (3, 3), activation="relu", padding='same')(Pool3)
    Conv4 = keras.layers.Conv2D(128, (3, 3), activation="relu", padding='same')(Conv4)
    Pool4 = keras.layers.MaxPooling2D((2, 2))(Conv4)

    # 다섯 번째 Convolution Block
    Conv5 = keras.layers.Conv2D(256, (3, 3), activation="relu", padding='same')(Pool4)
    Conv5 = keras.layers.Conv2D(256, (3, 3), activation="relu", padding='same')(Conv5)

    # 첫 번째 Upsampling Block
    Ups1 = keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2))(Conv5)
    Ups1 = keras.layers.Concatenate()([Ups1, Conv4])
    Ups1_conv = keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(Ups1)
    Ups1_conv = keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(Ups1_conv)

    # 두 번째 Upsampling Block
    Ups2 = keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(Ups1_conv)
    Ups2 = keras.layers.Concatenate()([Ups2, Conv3])
    Ups2_conv = keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(Ups2)
    Ups2_conv = keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(Ups2_conv)

    # 세 번째 Upsampling Block
    Ups3 = keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(Ups2_conv)
    Ups3 = keras.layers.Concatenate()([Ups3, Conv2])
    Ups3_conv = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(Ups3)
    Ups3_conv = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(Ups3_conv)

    # 네 번째 Upsampling Block
    Ups4 = keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding="same")(Ups3_conv)
    Ups4 = keras.layers.Concatenate()([Ups4, Conv1])
    Ups4_conv = keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same")(Ups4)
    Ups4_conv = keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same")(Ups4_conv)

    output_logit = keras.layers.Conv2D(1, (1, 1))(Ups4_conv)

    return keras.Model(inputs=input, outputs=output_logit)

In [ ]:
dr = DataReader()

In [ ]:
model = graph(128, 128)
model.load_weights('./unet_checkpoints')

In [ ]:
import matplotlib.pyplot as plt

images = dr.test_X
mask_y = dr.test_Y

plt.imshow(images[0])
plt.show()

In [ ]:
prediction = model.predict(images)[:, :, :, 0]
print(prediction.shape)
plt.imshow(prediction[0], cmap='gray')
plt.show()

In [ ]:
# print(prediction[0])

prediction[prediction < 0] = 0
print(prediction.shape)
plt.imshow(prediction[0], cmap='gray')
plt.show()

In [ ]:
# print(prediction[0])

pred_mask = (np.copy(images)*255.0).astype(np.uint8)[:, :, :, 0]
pred_mask[prediction > 0.5] = 255
print(pred_mask.shape)
plt.imshow(pred_mask[0], cmap='gray')
plt.show()

In [ ]:
mask= (np.copy(images)*255).astype(np.uint8)[:, :, :, 0]
mask[mask_y > 0.5] = 255
print(mask.shape)
plt.imshow(mask[0], cmap='gray')
plt.show()

In [ ]:
images = (images*255).astype(np.uint8)
template = np.copy(images)[:, :, :, 0]
print(template.shape)
plt.imshow(template[0], cmap='gray')
plt.show()

In [ ]:
mask = np.stack((template, mask, template), axis=3)
print(mask.shape)
plt.imshow(mask[0])
plt.show()

In [ ]:
pred_mask = np.stack((template, template, pred_mask), axis=3)
print(pred_mask.shape)
plt.imshow(pred_mask[0])
plt.show()

In [ ]:
new_canvas = np.concatenate((images[0], mask[0], pred_mask[0]), axis=1)
print(new_canvas.shape)
plt.imshow(new_canvas)
plt.figure()
plt.show()